In [ ]:
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Depth%20Anything/Predicting_depth_in_an_image_with_Depth_Anything.ipynb
# https://huggingface.co/docs/transformers/main/en/model_doc/depth_anything

#import cv2 is overkill.
from IPython.display import display
import numpy as np
from PIL import Image
import requests
import time
import torch
import torchvision
from urllib.parse import urlparse
# Requires 4.38.0 which is not yet released as of 2024-01-25. Otherwise,
# pip install git+https://github.com/huggingface/transformers
import transformers
print(f"Using transformers {transformers.__version__}")

# Equivalent of:
#pipe = transformers.pipeline(task="depth-estimation", model="LiheYoung/depth-anything-base-hf")
#result = pipe(image)["depth"]
processor = transformers.AutoImageProcessor.from_pretrained("nielsr/depth-anything-small")
model = transformers.AutoModelForDepthEstimation.from_pretrained("nielsr/depth-anything-small")

In [ ]:
def process_img(url):
    if url.startswith("http"):
        image = Image.open(requests.get(url, stream=True).raw)
    else:
        image = Image.open(url)
    display(image)
    
    start = time.time()
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    with torch.no_grad():
      predicted_depth = model(pixel_values).predicted_depth
    h, w = image.size[::-1]
    depth = torch.nn.functional.interpolate(predicted_depth[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    # Normalize as int8
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    depth_raster = depth.cpu().numpy().astype(np.uint8)
    # TODO(maruel): Colormap (palette) would be nice but not worth importing cv2 just
    # for that.
    #colored_depth = cv2.applyColorMap(depth_raster, cv2.COLORMAP_INFERNO)[:, :, ::-1]
    depth_img = torchvision.transforms.functional.to_pil_image(depth_raster, mode="L")
    display(depth_img)
    print(f"Inference in {time.time()-start:.1f}s")

In [ ]:
process_img("http://images.cocodataset.org/val2017/000000039769.jpg")

In [ ]:
process_img("https://maruel.ca/img/maruel.jpg")

In [ ]:
#process_img("../../tesla_wall.jpg")